In [114]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [115]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [116]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [117]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [118]:
STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  

STARTINGCSV=pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
10123,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,0,17,0,0,NSH,0,13:41,8475177,Calvin,de Haan
10676,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,-1,0,0,0,0,1,18,0,0,NSH,2,11:42,8480246,Nick,Perbix
4160,2023020003,VGK,H,2023-10-10,1,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,1,0,0,0,0,3,23,0,0,SEA,0,19:53,8478403,Jack,Eichel
8731,2023020001,NSH,R,2023-10-10,1,0,{'default': 'Predators'},{'default': 'Lightning'},1,1,0,0,0,0,2,20,0,0,TBL,0,13:43,8481704,Juuso,Parssinen
10216,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,1,23,0,0,NSH,0,14:01,8476822,Luke,Glendening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2023020637,COL,H,2024-01-10,0,0,{'default': 'Avalanche'},{'default': 'Golden Knights'},0,0,0,0,0,0,2,27,0,0,VGK,2,18:07,8471677,Jack,Johnson
599,2023020637,VGK,R,2024-01-10,0,0,{'default': 'Golden Knights'},{'default': 'Avalanche'},0,0,0,0,0,0,4,24,0,0,COL,0,23:15,8475913,Mark,Stone
1778,2023020635,MTL,R,2024-01-10,0,0,{'default': 'Canadiens'},{'default': 'Flyers'},0,1,0,0,0,0,0,19,0,0,PHI,0,14:06,8481058,Jesse,Ylönen
595,2023020637,VGK,R,2024-01-10,0,0,{'default': 'Golden Knights'},{'default': 'Avalanche'},0,0,0,0,0,0,0,18,0,0,COL,2,16:00,8475188,Brayden,McNabb


In [119]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Winnipeg,Jets,WPG
1,Vancouver,Canucks,VAN
2,Colorado,Avalanche,COL
3,Boston,Bruins,BOS
4,NY Rangers,Rangers,NYR
5,Florida,Panthers,FLA
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Toronto,Maple Leafs,TOR
9,Carolina,Hurricanes,CAR


In [120]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)

playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).shots.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)


playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,40,40,0.925000,0.463870,1,0.6,1,0.9
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,39,40,0.925000,0.463870,1,0.6,1,0.9
2,WPG,8475455,Brenden,Dillon,2023-12-27,0,CHI,38,40,0.925000,1.104736,1,0.6,1,0.9
3,WPG,8475455,Brenden,Dillon,2023-12-22,0,BOS,37,40,0.925000,1.104736,1,0.6,1,0.9
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,36,40,0.925000,0.463870,1,0.6,1,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,41,0.682927,0.463870,0,0.0,0,0.0
2306,SJS,8482667,William,Eklund,2024-01-09,2,TOR,4,41,1.512195,0.176996,0,2.6,0,2.3
2307,SJS,8482667,William,Eklund,2024-01-06,0,TOR,3,41,1.512195,1.104736,0,2.6,0,2.3
2308,SJS,8482667,William,Eklund,2024-01-04,0,WPG,2,41,1.512195,1.104736,0,2.6,0,2.3


In [121]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,40,40,0.925000,0.463870,1,0.6,1,0.9,22,22,1.272727
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,39,40,0.925000,0.463870,1,0.6,1,0.9,21,22,1.272727
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,36,40,0.925000,0.463870,1,0.6,1,0.9,20,22,1.272727
6,WPG,8475455,Brenden,Dillon,2023-12-16,1,COL,34,40,0.925000,0.463870,0,0.6,1,0.9,19,22,1.272727
7,WPG,8475455,Brenden,Dillon,2023-12-13,2,LAK,33,40,0.925000,0.176996,0,0.6,1,0.9,18,22,1.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,SJS,8481567,Henry,Thrun,2024-01-02,2,DET,1,41,0.585366,0.176996,0,0.0,0,0.0,1,11,1.636364
22309,SJS,8482222,Alexander,Barabanov,2024-01-04,3,WPG,2,41,0.682927,0.817861,0,0.0,0,0.0,2,12,1.666667
22310,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,41,0.682927,0.463870,0,0.0,0,0.0,1,12,1.666667
22311,SJS,8482667,William,Eklund,2024-01-09,2,TOR,4,41,1.512195,0.176996,0,2.6,0,2.3,2,29,1.758621


In [122]:
Teams

,City,TeamName,TeamID
0,Winnipeg,Jets,WPG
1,Vancouver,Canucks,VAN
2,Colorado,Avalanche,COL
3,Boston,Bruins,BOS
4,NY Rangers,Rangers,NYR
5,Florida,Panthers,FLA
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Toronto,Maple Leafs,TOR
9,Carolina,Hurricanes,CAR


In [123]:
import datetime
# todaysindex=datetime.datetime.today().weekday()+2
# if  todaysindex>6:
#     todaysindex-=7
#I think todaysindex is always 3
todaysindex=3

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,OTT,BUF
1,ANA,CAR
2,EDM,DET
3,LAK,FLA
4,SJS,MTL
5,VAN,PIT
6,NJD,TBL
7,SEA,WSH
8,TOR,NYI
9,NYR,STL


In [124]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots','LastTenAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['teamAbbrev'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='teamAbbrev', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent
0,BOS,8477956,David,Pastrnak,4.975000,5.8,6.2,5.218750,VGK
1,TBL,8476453,Nikita,Kucherov,4.261905,3.8,3.6,4.689655,NJD
2,NJD,8481559,Jack,Hughes,3.631579,0.0,2.0,4.600000,TBL
3,NYR,8478550,Artemi,Panarin,4.205128,5.4,4.9,4.303030,STL
4,TOR,8479318,Auston,Matthews,4.315789,3.6,5.0,4.281250,NYI
5,TOR,8477939,William,Nylander,4.315789,4.0,4.1,4.235294,NYI
6,OTT,8480801,Brady,Tkachuk,4.472222,5.4,5.0,4.233333,BUF
7,VGK,8478403,Jack,Eichel,4.121951,4.4,4.3,3.941176,BOS
8,WPG,8478398,Kyle,Connor,2.550000,0.0,0.0,3.900000,CHI
9,PIT,8477404,Jake,Guentzel,3.589744,5.0,4.5,3.741935,VAN


In [125]:
goalieSTARTINGCSV2023=pd.read_csv('GoalieLastYear2023Stats.csv')  
goalieSTARTINGCSV2024=pd.read_csv('GoalieThisYear2024Stats.csv')  

goalieSTARTINGCSV=pd.concat([goalieSTARTINGCSV2023, goalieSTARTINGCSV2024])
goalieSTARTINGCSV=goalieSTARTINGCSV[goalieSTARTINGCSV['gamesStarted']==1]
goalieSTARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,gamesStarted,decision,shotsAgainst,goalsAgainst,savePctg,shutouts,opponentAbbrev,pim,toi,playerId,firstName,lastName
586,2023020002,PIT,H,2023-10-10,0,0,{'default': 'Penguins'},{'default': 'Blackhawks'},1,L,35,3,0.914286,0,CHI,0,57:30,8477465,Tristan,Jarry
270,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,W,33,1,0.969697,0,SEA,0,60:00,8478499,Adin,Hill
672,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},1,W,31,3,0.903226,0,NSH,0,60:00,8477992,Jonas,Johansson
1098,2023020002,CHI,R,2023-10-10,0,0,{'default': 'Blackhawks'},{'default': 'Penguins'},1,W,41,2,0.951220,0,PIT,0,60:00,8475852,Petr,Mrazek
495,2023020001,NSH,R,2023-10-10,0,0,{'default': 'Predators'},{'default': 'Lightning'},1,L,33,4,0.878788,0,TBL,0,57:16,8477424,Juuse,Saros
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,2023020637,VGK,R,2024-01-10,0,0,{'default': 'Golden Knights'},{'default': 'Avalanche'},1,L,35,3,0.914286,0,COL,0,60:00,8480238,Jiri,Patera
98,2023020635,MTL,R,2024-01-10,0,0,{'default': 'Canadiens'},{'default': 'Flyers'},1,O,39,2,0.948718,0,PHI,0,65:00,8480051,Cayden,Primeau
103,2023020636,MIN,R,2024-01-10,0,0,{'default': 'Wild'},{'default': 'Stars'},1,L,34,7,0.794118,0,DAL,0,60:00,8482661,Jesper,Wallstedt
46,2023020635,PHI,H,2024-01-10,0,0,{'default': 'Flyers'},{'default': 'Canadiens'},1,W,19,2,0.894737,0,MTL,0,65:00,8481035,Samuel,Ersson


In [126]:
goalieStats=goalieSTARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','opponentAbbrev','gamesStarted','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['GameNumber'] = goalieStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
goalieStats['GP']=goalieStats.groupby(['playerId']).gamesStarted.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['playerId']).shotsAgainst.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgShotsAgainst']=goalieStats['AvgShotsAgainst'].rank(pct=True)*100

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgGoalsAgainst']=goalieStats['AvgGoalsAgainst'].rank(pct=True)*100

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgSavePctg']=goalieStats['AvgSavePctg'].rank(pct=True)*100

goalieStats['LastFiveFlag'] = np.where(goalieStats['GameNumber'] >= goalieStats['GP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['teamAbbrev','GP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['teamAbbrev'])['GP'].rank().astype(int)
summaryGoalieStats=summaryGoalieStats[['teamAbbrev','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','teamAbbrev':'GoalieteamAbbrev'})
summaryGoalieStats

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,WPG,1,Laurent,Brossoit,36.974444,9.439406,79.018961
9,WPG,2,Connor,Hellebuyck,35.325639,5.647156,87.262984
35,BOS,1,Linus,Ullmark,87.345425,39.447650,89.323990
53,BOS,1,Jeremy,Swayman,64.839242,18.507832,96.908491
71,VAN,2,Thatcher,Demko,61.665293,26.545754,80.626546
...,...,...,...,...,...,...,...
1075,CHI,2,Petr,Mrazek,80.708986,77.493817,41.797197
1100,CHI,1,Arvid,Soderblom,71.516900,98.392415,7.625721
1115,SJS,1,Kaapo,Kahkonen,89.447650,90.972795,19.703215
1133,SJS,2,Mackenzie,Blackwood,92.085738,93.322341,21.352020


In [127]:
full=todaysSkaters.merge(summaryGoalieStats[summaryGoalieStats['GPRank']==1], left_on='Opponent', right_on='GoalieteamAbbrev', how='left').drop(columns=['GoalieteamAbbrev','GPRank'])
full=full[['teamAbbrev', 'firstName', 'lastName', 'NoOutliersAvgSOG',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName','SeasonAvgSOG', 'playerId', 
       'RankAvgGoalsAgainst', 'RankAvgSavePctg']].rename(columns={"NoOutliersAvgSOG": "ExpectedSOG"}).round(2)

full.sort_values(by=['ExpectedSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(25)

,teamAbbrev,firstName,lastName,ExpectedSOG,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg
0,BOS,David,Pastrnak,5.22,5.8,6.2,VGK,99.79,Patera,4.97,8477956,99.79,49.67
1,TBL,Nikita,Kucherov,4.69,3.8,3.6,NJD,77.33,Daws,4.26,8476453,85.33,53.42
2,NJD,Jack,Hughes,4.60,0.0,2.0,TBL,23.17,Vasilevskiy,3.63,8481559,59.36,36.19
3,NYR,Artemi,Panarin,4.30,5.4,4.9,STL,30.34,Hofer,4.21,8478550,37.43,44.02
4,TOR,Auston,Matthews,4.28,3.6,5.0,NYI,85.99,Varlamov,4.32,8479318,32.56,84.67
5,TOR,William,Nylander,4.24,4.0,4.1,NYI,85.99,Varlamov,4.32,8477939,32.56,84.67
6,OTT,Brady,Tkachuk,4.23,5.4,5.0,BUF,42.66,Luukkonen,4.47,8480801,39.45,52.64
7,VGK,Jack,Eichel,3.94,4.4,4.3,BOS,87.35,Ullmark,4.12,8478403,39.45,89.32
8,VGK,Jack,Eichel,3.94,4.4,4.3,BOS,64.84,Swayman,4.12,8478403,18.51,96.91
9,WPG,Kyle,Connor,3.90,0.0,0.0,CHI,71.52,Soderblom,2.55,8478398,98.39,7.63


In [128]:
margin=full
margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['ExpectedSOG']

margin=margin[['teamAbbrev', 'firstName', 'lastName', 'ExpectedSOG',
       'LastFiveAvgShots', 'LastTenAvgShots', 'Opponent', 'RankAvgShotsAgainst',
        'GoalieLastName','SeasonAvgSOG', 'playerId', 
       'RankAvgGoalsAgainst', 'RankAvgSavePctg', 'lastfivemargin']]

margin.sort_values(by=['lastfivemargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,firstName,lastName,ExpectedSOG,LastFiveAvgShots,LastTenAvgShots,Opponent,RankAvgShotsAgainst,GoalieLastName,SeasonAvgSOG,playerId,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin
0,VAN,Elias,Pettersson,2.71,5.6,3.9,PIT,90.64,Nedeljkovic,2.76,8480012,24.61,91.63,2.89
1,VGK,Jonathan,Marchessault,2.86,5.0,4.3,BOS,87.35,Ullmark,3.37,8476539,39.45,89.32,2.14
2,VGK,Jonathan,Marchessault,2.86,5.0,4.3,BOS,64.84,Swayman,3.37,8476539,18.51,96.91,2.14
3,CGY,Jonathan,Huberdeau,1.77,3.6,2.2,ARI,72.84,Vejmelka,1.68,8476456,80.67,33.35,1.83
4,CGY,Elias,Lindholm,2.45,4.2,3.1,ARI,72.84,Vejmelka,2.66,8477496,80.67,33.35,1.75
5,LAK,Pierre-Luc,Dubois,1.88,3.6,2.7,FLA,6.06,Stolarz,1.86,8479400,2.84,85.61,1.72
6,PIT,Evgeni,Malkin,2.76,4.4,3.5,VAN,52.64,DeSmith,2.79,8471215,31.53,78.11,1.64
7,PIT,Kris,Letang,2.10,3.6,2.3,VAN,52.64,DeSmith,2.05,8471724,31.53,78.11,1.50
8,WPG,Nikolaj,Ehlers,2.94,4.4,3.8,CHI,71.52,Soderblom,2.90,8477940,98.39,7.63,1.46
9,NJD,Jonas,Siegenthaler,1.58,3.0,1.8,TBL,23.17,Vasilevskiy,1.21,8478399,59.36,36.19,1.42


In [129]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df